In [98]:
import pandas as pd
import os

In [101]:
new_dfs = []

for file in os.listdir('demo'):
    if file.endswith('csv'):
        curr_year = int(file.split('.')[0].replace('ACSST5Y', ''))
        df = pd.read_csv(f'demo/{file}', header=[1])
        if 'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Black or African American' not in set(df.columns):
            print('Column not found, adding columns')
            for column in df.columns:
                if column.startswith('Total!!Estimate!!'):
                    df[column.replace('Total!!Estimate!!', 'Estimate!!Total!!')] = df[column]
            for column in df.columns:
                if 'Estimate!!Total!!RACE AND HISPANIC OR LATINO ORIGIN' in column:
                    df[column.replace('Estimate!!Total!!RACE AND HISPANIC OR LATINO ORIGIN', 'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN')] = df[column]
        if 'Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino' not in df.columns:
            try:
                df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'] = df['Estimate!!Total!!Total population!!White alone, not Hispanic or Latino']
            except:
                df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'] = df['Estimate!!Total!!White alone, not Hispanic or Latino']

        new_df = pd.DataFrame()
        new_df['tract'] = df['Geographic Area Name']
        new_df['race_percent_white'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'].replace('-', 0).astype(float)
        new_df['race_percent_black'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Black or African American'].replace('-', 0).astype(float)
        new_df['race_percent_native'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!American Indian and Alaska Native'].replace('-', 0).astype(float)
        new_df['race_percent_asian'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Asian'].replace('-', 0).astype(float)
        new_df['race_percent_pacific_islander'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Native Hawaiian and Other Pacific Islander'].replace('-', 0).astype(float)
        new_df['race_percent_hispanic'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!White'].replace('-', 0).astype(float) - df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone, not Hispanic or Latino'].replace('-', 0).astype(float)
        new_df['race_percent_other'] = df['Estimate!!Total!!Total population!!RACE AND HISPANIC OR LATINO ORIGIN!!One race!!Some other race'].replace('-', 0).astype(float)
        new_df['income_percent_below_15K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$1 to $9,999 or loss"].replace('-', 0).astype(float) +\
                                            df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$10,000 to $14,999"].replace('-', 0).astype(float)
        new_df['income_percent_15K_35K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$15,000 to $24,999"].replace('-', 0).astype(float) +\
                                        df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$25,000 to $34,999"].replace('-', 0).astype(float)
        new_df['income_percent_35K_65K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$35,000 to $49,999"].replace('-', 0).astype(float) +\
                                        df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$50,000 to $64,999"].replace('-', 0).astype(float)
        new_df['income_percent_above_65K'] = df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$65,000 to $74,999"].replace('-', 0).astype(float) +\
                                            df[f"Estimate!!Total!!INDIVIDUALS' INCOME IN THE PAST 12 MONTHS (IN {curr_year} INFLATION-ADJUSTED DOLLARS)!!Population 15 years and over!!$75,000 or more"].replace('-', 0).astype(float)
        
        new_df['year'] = curr_year
        new_dfs.append(new_df)
new_df = pd.concat(new_dfs).sort_values(['year', 'tract']).reset_index(drop=True)

Column not found, adding columns
Column not found, adding columns
Column not found, adding columns


In [102]:
new_df

,tract,race_percent_white,race_percent_black,race_percent_native,race_percent_asian,race_percent_pacific_islander,race_percent_hispanic,race_percent_other,income_percent_below_15K,income_percent_15K_35K,income_percent_35K_65K,income_percent_above_65K,year
0,"Census Tract 1005, Allegheny County, Pennsylvania",28.6,65.7,0.0,0.0,0.0,0.0,0.0,36.9,29.6,20.2,4.7,2014
1,"Census Tract 1011, Allegheny County, Pennsylvania",79.3,11.5,0.0,2.5,0.0,2.8,0.4,33.1,32.8,18.2,5.0,2014
2,"Census Tract 1014, Allegheny County, Pennsylvania",82.3,11.8,0.3,1.2,0.0,2.1,0.4,25.1,28.5,24.0,13.4,2014
3,"Census Tract 1016, Allegheny County, Pennsylvania",0.4,98.9,0.0,0.0,0.0,0.7,0.0,37.7,25.3,13.0,3.1,2014
4,"Census Tract 1017, Allegheny County, Pennsylvania",20.4,76.1,1.0,0.0,0.0,1.1,0.0,46.1,22.1,15.0,6.0,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3589,Census Tract 9810; Allegheny County; Pennsylvania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022
3590,Census Tract 9811; Allegheny County; Pennsylvania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022
3591,Census Tract 9812; Allegheny County; Pennsylvania,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022
3592,Census Tract 9818; Allegheny County; Pennsylvania,71.2,21.8,0.0,4.7,0.0,1.0,0.0,48.3,27.2,8.5,4.6,2022


In [103]:
new_df.to_csv('total_demographics.csv', index=False)